In [16]:

import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
import matplotlib.pyplot as plt
%matplotlib inline
ps = nltk.PorterStemmer()
from nltk.tokenize import sent_tokenize, word_tokenize
from pprint import pprint

from string import punctuation

nltk.download('punkt')


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
  

[nltk_data] Downloading package punkt to /Users/selma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df=pd.read_csv('./npr.csv')

In [4]:
df.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [5]:
df.shape

(11992, 1)

# Count Vectorizer

In [12]:
cv=CountVectorizer(max_df=0.95,min_df=2, stop_words='english')

In [13]:
dtm=cv.fit_transform(df.Article)

In [14]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

# LDA using sklearn

In [17]:
LDA=LatentDirichletAllocation(n_components=5, random_state=0)

In [18]:
len(cv.get_feature_names())

54777

In [22]:
cv.get_feature_names()[40000]

'receiving'

In [23]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [24]:
len(LDA.components_)

5

In [27]:
LDA.components_


array([[2.68413523e+00, 7.96819990e+02, 2.00000843e-01, ...,
        2.00529488e-01, 2.23355814e-01, 2.00000188e-01],
       [1.10500977e+01, 4.88354017e+02, 2.00000374e-01, ...,
        2.00007505e-01, 2.00000191e-01, 2.00166933e-01],
       [4.06934502e+00, 2.53637019e+03, 2.00195124e-01, ...,
        2.01473200e-01, 2.00000225e-01, 2.00000095e-01],
       [4.87628755e+01, 2.47978044e+02, 3.19980316e+00, ...,
        6.19703855e+00, 2.17664350e+00, 2.19894488e+00],
       [2.84335465e+01, 1.39247776e+03, 2.00000500e-01, ...,
        2.00951260e-01, 2.00000269e-01, 2.00887902e-01]])

In [28]:
len(LDA.components_[0])

54777

In [29]:
Topic_0=LDA.components_[0]

In [30]:
Topic_0.argsort()

array([ 2475, 15888, 10658, ..., 33390, 53152, 42993])

In [31]:
Topic_0[15888]

0.20000003541969882

In [32]:
Topic_0.argsort()[::-10]

array([42993, 43238, 51917, ..., 26617, 40288, 21349])

In [34]:
Topic_0_top10=Topic_0.argsort()[-10:]

In [35]:
for i in Topic_0_top10:
    print (cv.get_feature_names()[i])

time
team
people
just
world
like
china
new
water
says


In [37]:
for index, topic in enumerate( LDA.components_):
    print ('Top 20 words for topic{}'.format(index))
    for i in topic.argsort()[-20:]:
        print (cv.get_feature_names()[i])
    print('********************************')    
        

Top 20 words for topic0
year
chinese
way
human
university
said
science
north
years
scientists
time
team
people
just
world
like
china
new
water
says
********************************
Top 20 words for topic1
vote
donald
party
political
court
just
states
election
republican
white
people
new
obama
house
state
campaign
clinton
president
said
trump
********************************
Top 20 words for topic2
patients
women
work
help
children
000
don
study
school
just
years
year
like
said
new
care
percent
health
people
says
********************************
Top 20 words for topic3
work
going
world
things
ve
make
music
don
life
years
way
really
know
think
new
time
people
just
says
like
********************************
Top 20 words for topic4
attack
just
military
killed
state
say
npr
time
according
years
war
country
government
reports
told
city
police
people
says
said
********************************


In [38]:
topic_results=LDA.transform(dtm)

In [39]:
topic_results.shape

(11992, 5)

In [41]:
topic_results[0].round(2)

array([0.  , 0.96, 0.  , 0.  , 0.04])

In [43]:
df.Article[0]

'In the Washington of 2016, even when the policy can be bipartisan, the politics cannot. And in that sense, this year shows little sign of ending on Dec. 31. When President Obama moved to sanction Russia over its alleged interference in the U. S. election just concluded, some Republicans who had long called for similar or more severe measures could scarcely bring themselves to approve. House Speaker Paul Ryan called the Obama measures ”appropriate” but also ”overdue” and ”a prime example of this administration’s ineffective foreign policy that has left America weaker in the eyes of the world.” Other GOP leaders sounded much the same theme. ”[We have] been urging President Obama for years to take strong action to deter Russia’s worldwide aggression, including its   operations,” wrote Rep. Devin Nunes,  . chairman of the House Intelligence Committee. ”Now with just a few weeks left in office, the president has suddenly decided that some stronger measures are indeed warranted.” Appearing 

In [44]:
topic_results[0].argmax()

1

In [45]:
df['topic']=topic_results.argmax(axis=1)

In [47]:
df.head(10)

,Article,topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",0
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,2
7,"I was standing by the airport exit, debating w...",3
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",3


In [48]:
df.topic.value_counts()

3    3526
1    2681
2    2211
4    2188
0    1386
Name: topic, dtype: int64

In [51]:
import pyLDAvis.sklearn

panel = pyLDAvis.sklearn.prepare(LDA, dtm, cv, mds='tsne')
panel

//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
3      10.844089  -55.716682       1        1  28.766826
1      41.403446  -86.304688       2        1  22.900112
2      26.546745 -128.632858       3        1  20.429975
4     -31.500032  -70.526352       4        1  16.801175
0     -12.336254 -109.429749       5        1  11.101912, topic_info=      Category          Freq          Term         Total  loglift  logprob
50426  Default  22587.000000         trump  22587.000000  30.0000  30.0000
42993  Default  45469.000000          says  45469.000000  29.0000  29.0000
22673  Default   8464.000000        health   8464.000000  28.0000  28.0000
9767   Default   7278.000000       clinton   7278.000000  27.0000  27.0000
38079  Default  11425.000000     president  11425.000000  26.0000  26.0000
42561  Default  28489.000000          said  28489.000000  25.0000  25.0000
37374  Default   4959.000000        police   4959.000000  24.0000  24.0000
7900   Default   5557.000000      campaign   5557.000000  23.0000  23.0000
34145  Default   4970.000000         obama   4970.000000  22.0000  22.0000
36310  Default   7929.000000       percent   7929.000000  21.0000  21.0000
16123  Default   3591.000000      election   3591.000000  20.0000  20.0000
8149   Default   5529.000000          care   5529.000000  19.0000  19.0000
41009  Default   3667.000000    republican   3667.000000  18.0000  18.0000
53152  Default   3497.000000         water   3497.000000  17.0000  17.0000
9189   Default   2401.000000         china   2401.000000  16.0000  16.0000
32729  Default   3609.000000         music   3609.000000  15.0000  15.0000
15010  Default   2967.000000        donald   2967.000000  14.0000  14.0000
52782  Default   2949.000000          vote   2949.000000  13.0000  13.0000
52786  Default   2772.000000        voters   2772.000000  12.0000  12.0000
38081  Default   2687.000000  presidential   2687.000000  11.0000  11.0000
35959  Default   2320.000000      patients   2320.000000  10.0000  10.0000
43714  Default   2555.000000        senate   2555.000000   9.0000   9.0000
41011  Default   2620.000000   republicans   2620.000000   8.0000   8.0000
42749  Default   2341.000000       sanders   2341.000000   7.0000   7.0000
23698  Default   6734.000000         house   6734.000000   6.0000   6.0000
13436  Default   2300.000000     democrats   2300.000000   5.0000   5.0000
47218  Default   3604.000000         study   3604.000000   4.0000   4.0000
46581  Default   9955.000000         state   9955.000000   3.0000   3.0000
35843  Default   3506.000000         party   3506.000000   2.0000   2.0000
13431  Default   2541.000000    democratic   2541.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
18833   Topic5    518.858036          fish    689.993921   1.9130  -6.8746
9736    Topic5    905.956163       climate   1369.710147   1.7847  -6.3172
43232   Topic5   1272.779875       science   2203.297718   1.6493  -5.9772
48690   Topic5   1634.399186          team   3106.701275   1.5558  -5.7272
48738   Topic5    820.175402    technology   1281.154474   1.7521  -6.4167
20208   Topic5    970.261049          game   1665.122759   1.6580  -6.2486
33864   Topic5   1288.017281         north   2774.561200   1.4307  -5.9653
35751   Topic5    711.570235          park   1195.138567   1.6795  -6.5587
34435   Topic5    706.147428           oil   1200.430296   1.6674  -6.5664
42993   Topic5   7461.854476          says  45469.002679   0.3908  -4.2086
54102   Topic5   1853.618518         world   8255.961803   0.7043  -5.6013
17914   Topic5    791.101288      facebook   1625.043923   1.4782  -6.4528
23826   Topic5   1090.797125         human   3134.567065   1.1425  -6.1315
51417   Topic5   1222.255723    university   4724.679928   0.8460  -6.0177
33390   Topic5   2102.541945           new  17309.110569   0.0900  -5.4753
28659   Topic5   1895.056915     